In [31]:
import os
import numpy as np
import importlib

In [32]:
feature_number = 105

In [33]:
X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
print(X.shape)
print(y.shape)

(393345, 105)
(393345,)


In [34]:
import split

In [35]:
X, y = split.delete_question_mark(X, y)
print(X.shape)
print(y.shape)

(392309, 105)
(392309,)


In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.fit_transform(X_train)

In [39]:
from sklearn.decomposition import PCA
pca = PCA(n_components= 0.999)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)
X_pca.shape

(313847, 82)

In [40]:
X_W, y_W = split.split_wake_and_asleep(X_pca, y_train)
print(X_W.shape)
print(y_W.shape)

(313847, 82)
(313847,)


In [41]:
import intergrating_model

In [42]:
importlib.reload(intergrating_model)
W_model = intergrating_model.generate_model(X_W, y_W)

svm finished
log finished
NB finished
SVM: 0.9558069141309543
Logistic: 0.9559343635494663
Naive Bayes: 0.550007965588657


In [59]:
import test
X_test_W, y_test_W = split.split_wake_and_asleep(X_test, y_test)
test.test_test(X_test_W, y_test_W, pca, W_model)

Class 0: Precision: 0.727, Recall: 0.951
Class 1: Precision: 0.453, Recall: 0.102
mf1 score: 0.557
acc_score: 0.710
kappa_score: 0.831


In [43]:
X_REM, y_REM = split.split_eye_movement(X_pca, y_train)
REM_model = intergrating_model.generate_model(X_REM, y_REM)

svm finished
log finished
NB finished
SVM: 0.8754716981132076
Logistic: 0.8772176851591101
Naive Bayes: 0.788791889608561


In [44]:
X_N, y_N = split.split_deep_sleep(X_pca, y_train)
N_model = intergrating_model.generate_model(X_N, y_N)

svm finished
log finished
NB finished
SVM: 0.9651766402307138
Logistic: 0.9647440519105984
Naive Bayes: 0.9180966113914925


In [45]:
X_12, y_12 = split.split_1_2(X_pca, y_train)
model_12 = intergrating_model.generate_model(X_12, y_12)

svm finished
log finished
NB finished
SVM: 0.865979381443299
Logistic: 0.8682170542635659
Naive Bayes: 0.7559338288180293


In [46]:
X_34, y_34 = split.split_3_4(X_pca, y_train)
model_34 = intergrating_model.generate_model(X_34, y_34)

svm finished
log finished
NB finished
SVM: 0.871039056742815
Logistic: 0.8680913780397936
Naive Bayes: 0.7487103905674282


In [47]:
importlib.reload(intergrating_model)
y_pred = intergrating_model.int_model(X_test, pca, W_model, REM_model, N_model, model_12, model_34)

100%|███████████████████████████████████████████████████████████████████████████| 78462/78462 [04:47<00:00, 273.32it/s]


In [56]:
np.sum(y_pred == 0) + np.sum(y_pred == 5)

78462

In [57]:
y_pred.shape

(78462,)

In [48]:
import scoring

In [49]:
importlib.reload(scoring)
scoring.score(y_pred, y_test, 6)

Class 0: Precision: 0.727, Recall: 0.951
Class 1: Precision: 0.000, Recall: 0.000
Class 2: Precision: 0.000, Recall: 0.000
Class 3: Precision: 0.000, Recall: 0.000
Class 4: Precision: 0.000, Recall: 0.000
Class 5: Precision: 0.058, Recall: 0.060
mf1 score: nan
acc_score: 0.684
kappa_score: 0.821


E:\2023 Fall\IML\project\iml-project\scoring.py:26: RuntimeWarning: invalid value encountered in scalar divide
  pre += TP / (TP + FP)
